### Try different optimizer to finetune the learningrate better

In [ ]:
#%pylab inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(precision = 6)
import datetime
matplotlib.rcParams['figure.figsize'] = [6, 4]
plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['image.interpolation']='catrom'
from vampnet import data_generator

In [ ]:
import tensorflow as tf
import scipy.io
import binascii
# import pyemma as py
import mdshare
import vampnet
import mdtraj as md

In [ ]:
traj_whole = np.load('/group/ag_cmb/scratch/deeptime_data/alanine_data/coordinates.npy')
dihedral = np.load('/group/ag_cmb/scratch/deeptime_data/alanine_data/dihedral.npy')

# traj_whole = np.concatenate([np.sin(dihedral), np.cos(dihedral)], axis = 1)

traj_data_points, input_size = traj_whole.shape

In [ ]:
batch_size = 8000
batch_size_gen = 1500
#batch_size_latent = 1024
tau = 8

train_ratio = 0.9

lr_koop = 1e-5
lr_gen = 1e-4

output_size = 10

nodes = [100, 100, 100, 100, 100, 100, output_size]

load_model = False
load_gen_model = False

res_block_len = 5
regularizer = 1e-10
dropout = 0.#0.5# 0.2

random_size = 6

sample_training_data_for_gen_before = True

col_gen = 'gen'
nodes_gen = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, input_size]


dict_variables = {}

In [ ]:
# Select the state to be filtered out
# 0 top left, 1 top right, 2 bottom left, 3 bottom right
state_to_be_filtered = 4


# samples of the original trajectory which are closest to the center of each state
state_centers = [8108, 133869, 0, 19678, 174924, 94042]

# Filters used for each state
filter_boxes = np.array([
    [[-np.pi, -1.8],[ 1.8, -2.0]],
    [[  -1.8, -0.3],[ 1.4, -2.0]],
    [[-np.pi, -1.8],[-2.0,  1.8]],
    [[  -1.9, -0.3],[-2.0,  1.4]],
    [[     0,    2],[  -2,    2]],
    [[     0,    2],[   2,   -2]]
])
state_filt_index = state_centers[state_to_be_filtered]
filtbox = filter_boxes[state_to_be_filtered]


mplt_filt1 = [[filtbox[0,0],filtbox[1,0]], filtbox[0,1] - filtbox[0,0], filtbox[1,1] - filtbox[1,0]]
mplt_filt2 = None

if filtbox[1,0] > filtbox[1,1]:
    mplt_filt1 = [[filtbox[0,0],-np.pi], filtbox[0,1] - filtbox[0,0], filtbox[1,1] + np.pi]
    mplt_filt2 = [[filtbox[0,0],filtbox[1,0]], filtbox[0,1] - filtbox[0,0], np.pi - filtbox[1,0]]

def get_filter(tau, dih, filt_shape, circle = False):
            
    dht_1 = np.array(dih[:-tau].T)
    dht_2 = np.array(dih[tau:].T)
    
    if circle:
    
        center = [-2.5,0]
        radius = [0.8, 1]
        
        for dht in [dht_1, dht_2]:
            for d, c, r in zip(dht, center, radius):
                d -= c
                d /= r

        cond_1 = dht_1[0]**2 + dht_1[1]**2 > 1
        cond_2 = dht_2[0]**2 + dht_2[1]**2 > 1
        
        indexes_filt = np.where(np.all([cond_1, cond_2], axis = 0))[0]
    
        return indexes_filt
        
    else:
        
        xlim = filt_shape[0]
        ylim1 = filt_shape[1]
        
        if ylim1[0] < ylim1[1]:
            ylim2 = [0,0]
        else:
            ylim2  = [-np.pi, ylim1[1]]
            ylim1[1] = np.pi
        
        conds = []
        
        for ylim in [ylim1, ylim2]:
            for index_dht, dht in enumerate([dht_1, dht_2]):

                cond_x = np.any([dht[0] < xlim[0], dht[0] > xlim[1]], axis = 0)
                cond_y = np.any([dht[1] < ylim[0], dht[1] > ylim[1]], axis = 0)

                conds.append(np.any([cond_x, cond_y], axis = 0))
        
        
        indexes_filt = np.where(np.all(conds, axis = 0))[0]

        return indexes_filt

In [ ]:
filter_data = False

if filter_data:
    length_data_whole = traj_data_points - tau

    indexes_filt = get_filter(tau, dihedral, filtbox)

    length_data = indexes_filt.shape[0]

    traj_ord = traj_whole[:-tau][indexes_filt]
    traj_ord_lag = traj_whole[tau:][indexes_filt]

    shuffle_index = np.arange(length_data)

    np.random.shuffle(shuffle_index)

    traj = traj_ord[shuffle_index]
    traj_lag = traj_ord_lag[shuffle_index]

    dihedral_filt = dihedral[:-tau][indexes_filt]


else:
    length_data = traj_data_points - tau

    traj_ord = traj_whole[:-tau]
    traj_ord_lag = traj_whole[tau:]

    shuffle_index = np.arange(length_data)

    np.random.shuffle(shuffle_index)

    traj = traj_ord[shuffle_index]
    traj_lag = traj_ord_lag[shuffle_index]

    dihedral_filt = dihedral[:-tau]

In [ ]:
fig, ax = plt.subplots()

range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

hist_im = ax.hist2d(dihedral[:,0], dihedral[:,1], bins=300, range = range_mat, norm=matplotlib.colors.LogNorm(), cmap = plt.cm.viridis)
ax.set_xlim(-np.pi,np.pi)
ax.set_ylim(-np.pi,np.pi)

if filter_data:
    ax.scatter(dihedral[state_filt_index,0], dihedral[state_filt_index,1], s=40, c='C3')
    e = matplotlib.patches.Rectangle(*mplt_filt1, color='C1')
    e.set_clip_box(ax.bbox)
    e.set_alpha(0.2)
    ax.add_artist(e)

    if mplt_filt2:
        e2 = matplotlib.patches.Rectangle(*mplt_filt2, color='C1')
        e2.set_clip_box(ax.bbox)
        e2.set_alpha(0.2)
        ax.add_artist(e2)
    
font = {'fontsize': 14}
# ax.set_xticks([-3,0,3])
# # ax.set_yticks([-3,0,3])
# ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
# ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
plt.colorbar(hist_im[-1], ax = ax)
# fig.savefig('training_data_overlay_{}state.pdf'.format(state_to_be_filtered), bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots()

range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

hist_im = ax.hist2d(dihedral_filt[:,0], dihedral_filt[:,1], bins=300, range = range_mat, norm=matplotlib.colors.LogNorm(), cmap = plt.cm.viridis)
if filter_data:
    ax.scatter(dihedral[state_filt_index,0], dihedral[state_filt_index,1], s=40, c='C3')
    
ax.set_xlim(-np.pi,np.pi)
ax.set_ylim(-np.pi,np.pi)
font = {'fontsize': 14}
ax.set_xticks([-3,0,3])
ax.set_yticks([-3,0,3])
ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
plt.colorbar(hist_im[-1], ax = ax)
fig.savefig('training_data_removed_{}state.pdf'.format(state_to_be_filtered), bbox_inches='tight')
plt.show()

In [ ]:
length_train = int(np.floor(length_data * train_ratio))
length_vali = length_data - length_train

traj_data_train = traj[:length_train]
traj_data_train_lag = traj_lag[:length_train]

traj_data_valid = traj[length_train:]
traj_data_valid_lag = traj_lag[length_train:]

# Input of the first network
X1_train = traj_data_train.astype('float32')
X2_train  = traj_data_train_lag.astype('float32')

# Input for validation
X1_vali = traj_data_valid.astype('float32')
X2_vali = traj_data_valid_lag.astype('float32')

In [ ]:
def create_layer(in_size, out_size, col):
    w = tf.get_variable(shape = [in_size, out_size], initializer=tf.contrib.layers.xavier_initializer(), name='W', collections = [col])
    b = tf.get_variable(initializer=tf.constant(0., shape = [out_size]), name='b', collections = [col])
    return w,b

In [ ]:
def layers_func(nodes, acti = tf.nn.elu, output_acti = tf.nn.softmax):
    layers_list = [tf.layers.BatchNormalization()]
    for node in nodes[:-1]:
        layers_list.append(tf.layers.Dense(node, activation=acti))
    layers_list.append(tf.layers.Dense(nodes[-1], activation=output_acti))
    return layers_list

In [ ]:
def chi(x_t, layers, name='chi'):
    
    with tf.variable_scope(name):
        l = tf.identity(x_t, name = 'input')
        
        for layer in layers:
            l = layer(l)
        
    return l

In [ ]:
def gamma(x_t, layers, name='gamma'):
    
    
    with tf.variable_scope(name):
        l = tf.identity(x_t, name = 'input')
        
        for layer in layers:
            l = layer(l)
        
    return l

In [ ]:
def layers_generator(nodes, acti=tf.nn.elu, res_dept=3, drop=0.):
    layers_list = []
    for node in nodes[:-1]:
        for res in range(res_dept):
            layers_list.append(tf.layers.BatchNormalization())
            layers_list.append(tf.layers.Dense(node, activation=acti))
            layers_list.append(tf.layers.Dropout(rate=drop))
    layers_list.append(tf.layers.BatchNormalization())
    layers_list.append(tf.layers.Dense(nodes[-1]))
    return layers_list

In [ ]:
def generator(x_t, layers, fac_res, block_len=3, acti=tf.nn.elu, name='generator'):
    
    with tf.variable_scope(name):
        batch_size = tf.shape(x_t)[0]
        rand_input = tf.random_normal(shape = [batch_size, random_size], name = 'random_input')
        # find the state at t
        rand_num = tf.random_uniform(tf.expand_dims(batch_size, axis=0), minval=0, maxval=1)
        state = output_size - tf.reduce_sum(tf.to_int32(tf.cumsum(x_t, axis=1) >= rand_num), axis=1)
        
#         print(state)
        x_sample = tf.gather_nd(tf.eye(output_size), tf.expand_dims(state,axis=1))
        
        
        
        l = tf.concat([x_t, rand_input], axis = -1, name = 'input')
        
        counter = 0
#         l = layers[counter](l) # first batchnorm
#         counter += 1
        
        number_res_layers = (len(layers)-2) // block_len // 3 # -2 because output and first batchnorm
        
        for res_layer in range(number_res_layers):
            for _ in range(3): #  batchnorm, Dense, drop
                l = layers[counter](l)
                counter += 1
            l_res = tf.identity(l)
            for _ in range(block_len-1): # res block same size
                for _ in range(3):  # batchnorm, Dense, drop 
                    l_res = layers[counter](l_res)
                    counter += 1
            
            # combine res
            l = acti(tf.add(fac_res * l, l_res))
        l = layers[counter](l) # batchnorm
        output = layers[counter+1](l)
        
        return output
                

In [ ]:
tf.reset_default_graph()

dict_variables = {}

In [ ]:
layers_chi = layers_func(nodes)
layers_gamma = layers_func(nodes, output_acti=tf.nn.relu)


In [ ]:
x_t = tf.placeholder(tf.float32, shape = [None, input_size], name = 'input_t')
x_tau = tf.placeholder(tf.float32, shape = [None, input_size], name = 'input_tau')
keep_prob_drop = tf.placeholder_with_default(1., shape = (), name = 'prob_drop')

layers_gen = layers_generator(nodes_gen, drop=1.-keep_prob_drop)

chi_out = chi(x_t, layers_chi)
gamma_out = gamma(x_tau, layers_gamma)

In [ ]:
gen_chi_tau = tf.placeholder(tf.float32, shape = (None, output_size), name = 'input_gen')
gen_chi_tau2 = tf.placeholder(tf.float32, shape = (None, output_size), name = 'input_gen')
gen_x_tau = tf.placeholder(tf.float32, shape = (None, input_size), name = 'label_gen')
k_tf = tf.placeholder(tf.float32, shape = (output_size, output_size), name = 'koop_op_gen')
fac_res_tf = tf.placeholder_with_default(0., shape=(), name='factor_res')



gen_x1 = generator(gen_chi_tau, layers_gen, fac_res_tf, res_block_len)
gen_x2 = generator(gen_chi_tau2, layers_gen, fac_res_tf, res_block_len)

In [ ]:
var_chi = []
var_gamma = []
for i in range(len(layers_chi)):
    var_chi += layers_chi[i].variables
    var_gamma += layers_gamma[i].variables
var_gen = []
for i in range(len(layers_gen)):
    var_gen += layers_gen[i].variables

In [ ]:
def energy_match_loss(x_tau, y, y_bar):

    norm_01 = tf.norm(x_tau - y, axis = 1)
    norm_02 = tf.norm(x_tau - y_bar, axis = 1)
    norm_12 = tf.norm(y - y_bar, axis = 1)
    batch_size_temp = tf.shape(x_tau)[0]
    ret = 1./tf.cast(batch_size_temp, tf.float32)*tf.reduce_sum(norm_01+norm_02-norm_12, axis=0)
    return ret 

def vampe_loss(chi, gamma):
    b = tf.to_float(tf.shape(chi)[0])
    c00 = 1/b*(tf.transpose(chi) @ chi)
    c11 = 1/b*(tf.transpose(gamma) @ gamma)
    c01 = 1/b*(tf.transpose(chi) @ gamma)

    gamma_dia_inv = tf.diag(1/(tf.reduce_mean(gamma, axis = 0)+ regularizer))  # add something so no devide by zero

    first_term = c00 @ gamma_dia_inv @ c11 @ gamma_dia_inv
    second_term = 2 * (c01 @ gamma_dia_inv)
    vampe_arg = first_term - second_term
    vampe = tf.trace(vampe_arg)
    
    return vampe

In [ ]:
with tf.name_scope('Losses'):
    
    with tf.name_scope('generator'):
        loss_gen = energy_match_loss(gen_x_tau, gen_x1, gen_x2)

    with tf.name_scope('koopman'):
        loss_vampe = vampe_loss(chi_out, gamma_out)

In [ ]:
with tf.name_scope('optimizers'):
    with tf.name_scope('koopman'):
        var_koop = var_chi + var_gamma
        opt = tf.train.AdamOptimizer(learning_rate=lr_koop)
        gvs = opt.compute_gradients(loss_vampe, var_list = var_koop)
        crapped_gvs = [(tf.clip_by_value(grad, -1.,1.), var) for grad, var in gvs]
        solver_koop = opt.apply_gradients(crapped_gvs)
        var_adam_koop = opt.variables()
    
    with tf.name_scope('gam_opt'):
        opt = tf.train.AdamOptimizer(learning_rate=lr_koop)
        gvs = opt.compute_gradients(loss_vampe, var_list = var_gamma)
        crapped_gvs = [(tf.clip_by_value(grad, -1.,1.), var) for grad, var in gvs]
        solver_gamma = opt.apply_gradients(crapped_gvs)
        var_adam_gam = opt.variables()
    
    with tf.name_scope('generator'):
        lr_tensor = tf.placeholder(tf.float32, shape = (), name = 'leanrning_rate')
#         var_gen = tf.get_collection(col_gen)
        
        opt = tf.train.AdamOptimizer(learning_rate=lr_gen)
        gvs = opt.compute_gradients(loss_gen, var_list = var_gen)
        crapped_gvs = [(tf.clip_by_value(grad, -1.,1.), var) for grad, var in gvs]
        solver_gen = opt.apply_gradients(crapped_gvs)
        var_adam_gen = opt.variables()
        
        opt_sg = tf.train.GradientDescentOptimizer(learning_rate=lr_tensor)
        gvs_sg = opt_sg.compute_gradients(loss_gen, var_list = var_gen)
        crapped_gvs_sg = [(tf.clip_by_value(grad, -1.,1.), var) for grad, var in gvs_sg]
        solver_gen_sg = opt.apply_gradients(crapped_gvs_sg)
        var_adam_gen += opt.variables()

In [ ]:
with tf.name_scope('initializers'):
    init_var_koop = tf.variables_initializer(var_koop)
    init_var_gamma = tf.variables_initializer(var_gamma)
    init_global_var = tf.global_variables_initializer()
    init_var_gen = tf.variables_initializer(var_gen)
    
    init_adam_koop = tf.variables_initializer(var_adam_koop)
    init_adam_gen = tf.variables_initializer(var_adam_gen)
    init_adam_gam = tf.variables_initializer(var_adam_gam)
    


In [ ]:
sess = tf.Session()
def reset_weights_koop():
    sess.run([init_var_koop, init_adam_koop])

reset_weights_koop()

writer = tf.summary.FileWriter(logdir = './tensorboard_log/', graph = tf.get_default_graph())

In [ ]:
def get_mini_batch(batchsize, index):
    
    start = index*batchsize
    end = (index+1)*batchsize
    
    if(end < length_train):
        batch_traj = X1_train[start : end]
        batch_traj_lag = X2_train[start:end]
        
    else:
        batch_traj = X1_train[start:]
        batch_traj_lag = X2_train[start:]
    
    return batch_traj, batch_traj_lag

In [ ]:
reset_weights_koop()

In [ ]:
trainings_completed = 0
batches = np.ceil(length_train/batch_size).astype('int')

In [ ]:
saver_koop = tf.train.Saver(var_koop+var_adam_koop)
if load_model:
    if filter_data:
        saver_koop.restore(sess, "./saved_models/working_alavamp_filter_state{}.ckpt".format(state_to_be_filtered))
    else:
        saver_koop.restore(sess, '/group/ag_cmb/scratch/deeptime_data/data_nips/ML_koopman_working_alavamp.ckpt')

if False:
    save_path = saver_koop.save(sess, "./saved_models/working_alavamp_filter_state{}.ckpt".format(state_to_be_filtered))

In [ ]:
best_vamp_score = 0.
if not load_model:
    while best_vamp_score > -2.95:
        reset_weights_koop()
        epochs = 801
        plot_save_every = 25
        offset = trainings_completed

        # compute distribution of training data
        output_nodes_list = [solver_koop, 
                             loss_vampe,
                                  ]
        flag = False
        for run in range(epochs):
            if flag:
                break    
            loss_curr = []
            loss_vali_curr = []

            for batch in range(batches):
                # Get data for minibatch
                batch_traj, batch_traj_lag = get_mini_batch(batch_size, batch)


                feed_dict_D = {x_t: batch_traj,
                               x_tau: batch_traj_lag,
                               keep_prob_drop : (1.-dropout)
                              }

                # Create Input for Training D
                # Train Dyn
                _, loss_temp1 = sess.run(output_nodes_list,
                                                feed_dict=feed_dict_D)
                loss_curr.append(loss_temp1)
                # Write to Summary
            loss_vali_temp = sess.run(loss_vampe, feed_dict={x_t : X1_vali,
                                                           x_tau :  X2_vali,
                                                           keep_prob_drop : 1.})

        #         if (np.isnan(loss_temp1)):
        #             print('found NAN in batch {}'.format(batch))
        #             flag = True
        #             break

                


            loss_np = np.mean(loss_curr)
            loss_vali = loss_vali_temp

        #     n1_all.append(liste_n1)
        #     n2_all.append(liste_n2)
        #     n12_all.append(liste_n12)
            # Create a new Summary object with your measure
            summary = tf.Summary()
            summary.value.add(tag="loss", simple_value=loss_vali)

            # Add it to the Tensorboard summary writer
            # Make sure to specify a step parameter to get nice graphs over time
            writer.add_summary(summary, run+offset)  

            if run % plot_save_every == 0:
                print('Iter: {}'.format(run))
                print('Loss: {:.4}'.format(loss_np))
                print('Loss vali: {:.4}'.format(loss_vali))
                print()

        trainings_completed += epochs
        pred = sess.run(chi_out, feed_dict = {x_t: traj_whole, keep_prob_drop : 1.})
        fig, ax = plt.subplots()

        range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

        maxi_train = np.max(pred, axis= 1)
        coor_train = np.zeros_like(pred)
        for i in range(output_size):
            coor_train = np.where(pred[:,i]== maxi_train)[0]
            ax.scatter(dihedral[coor_train,0], dihedral[coor_train,1], s=1)

        ax.set_xlim(-np.pi,np.pi)
        ax.set_ylim(-np.pi,np.pi)

        if filter_data:
            e = matplotlib.patches.Rectangle(*mplt_filt1, color='k')
            e.set_clip_box(ax.bbox)
            e.set_alpha(0.2)
            ax.add_artist(e)

        font = {'fontsize': 14}
        ax.set_xticks([-3,0,3])
        ax.set_yticks([-3,0,3])
        ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
        ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
        ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
        ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
        plt.show()
        best_vamp_score = loss_vali

In [ ]:
pred = sess.run(chi_out, feed_dict = {x_t: traj_whole, keep_prob_drop : 1.})

In [ ]:
fig, ax = plt.subplots()

range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

maxi_train = np.max(pred, axis= 1)
coor_train = np.zeros_like(pred)
for i in range(output_size):
    coor_train = np.where(pred[:,i]== maxi_train)[0]
    ax.scatter(dihedral[coor_train,0], dihedral[coor_train,1], s=1)
    
ax.set_xlim(-np.pi,np.pi)
ax.set_ylim(-np.pi,np.pi)

if filter_data:
    e = matplotlib.patches.Rectangle(*mplt_filt1, color='k')
    e.set_clip_box(ax.bbox)
    e.set_alpha(0.2)
    ax.add_artist(e)
    
font = {'fontsize': 14}
ax.set_xticks([-3,0,3])
ax.set_yticks([-3,0,3])
ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
plt.show()


In [ ]:
def estimate_K_vampe(traj_ord_lag):

    chi_xtau = sess.run(chi_out, feed_dict = {x_t: traj_ord_lag, keep_prob_drop : 1.})
    gamma_xtau = sess.run(gamma_out, feed_dict = {x_tau: traj_ord_lag, keep_prob_drop : 1.})

    gamma_mean = 1/(np.mean(gamma_xtau, axis = 0) + regularizer)

    K = np.empty((output_size, output_size))

    for i in range(output_size):
        for j in range(output_size):
            K[i,j] = np.mean(gamma_mean[i]*gamma_xtau[:,i]*chi_xtau[:,j], axis = 0)
            
    return K

In [ ]:
K = estimate_K_vampe(traj_ord_lag)

eigv, eigw = np.linalg.eig(K)

eig_sort = np.argsort(eigv)[::-1]

eigv = eigv[eig_sort]
eigw = eigw[:, eig_sort]

# Generative Model

In [ ]:
def reset_weights_gen():
    sess.run([init_var_gen, init_adam_gen])

reset_weights_gen()

In [ ]:
chi_ord = sess.run(chi_out, feed_dict = {x_t: traj_ord, keep_prob_drop : 1.})
chi_ord_lag = sess.run(chi_out, feed_dict = {x_t: traj_ord_lag, keep_prob_drop : 1.})

chi_shuffle = chi_ord[shuffle_index]
chi_shuffle_lag = chi_ord_lag[shuffle_index]

chi_data_train = chi_shuffle[:length_train]
chi_data_train_lag = chi_shuffle_lag[:length_train]

chi_data_valid = chi_shuffle[length_train:]
chi_data_valid_lag = chi_shuffle_lag[length_train:]

# Input of the first network
chi_X1_train = chi_data_train.astype('float32')
chi_X2_train  = chi_data_train_lag.astype('float32')

# Input for validation
chi_X1_vali = chi_data_valid.astype('float32')
chi_X2_vali = chi_data_valid_lag.astype('float32')

In [ ]:
def sample_from_dist(dist, arg_max = False, cumsum = False):
    sample_num, sample_size = dist.shape
    drawn_values = np.random.rand(sample_num)
    sample_retval = np.zeros_like(dist)
    if cumsum:
        sample_cumsum = dist
    else:
        sample_cumsum = np.cumsum(dist, axis = 1)
    
    sample_diff = sample_cumsum - np.tile(drawn_values,(sample_size,1)).T
    sample_sign = np.sign(sample_diff)
    sample_argmax = np.argmax(sample_sign, axis = 1)
    
    if arg_max: 
        return sample_argmax
    sample_retval[np.arange(sample_num), sample_argmax] = 1
    
    return sample_retval

def get_mini_batch_gen(batchsize, index):
    
    start = index*batchsize
    end = (index+1)*batchsize
        
    if(end < length_train):
            
        
        batch_traj = chi_X1_train[start : end]
        
        batch_traj_lag = X2_train[start : end]
        
    else:
        
        batch_traj = chi_X1_train[start:]
       
        
        batch_traj_lag = X2_train[start:]
        
        
    
    return batch_traj, batch_traj_lag

In [ ]:
trainings_completed_gen = 0

In [ ]:
sess.run([init_var_gen, init_adam_gen, init_adam_gam]);

In [ ]:
# batch_size_gen = 256 # not over 2100 because the lowest populated is
batches = np.ceil(length_train/batch_size_gen).astype('int')
best_value = 1.

### Sample training data before 

In [ ]:
saver = tf.train.Saver(var_koop + var_gen)

if load_gen_model:
    saver.restore(sess, "./saved_models/best_ala_gen_model_equal.ckpt")
    
    if False:
        save_path = saver.save(sess, "./saved_models/best_ala_gen_model_equal.ckpt")

### Factor back to old version

In [ ]:
epochs = 501
plot_save_every = 25
offset = trainings_completed_gen

stochastic_gradient = False
lr_gentemp = lr_gen / 100000.

# compute distribution of training data
if stochastic_gradient:
    output_nodes_list = [solver_gen_sg, 
#                      loss_gen,
                          ]
else:
    output_nodes_list = [solver_gen, 
#                      loss_gen,
                          ]
flag = False
fac_res_temp = 1.
fac_smaller = 1. /( (epochs // 100) - 1) # so it is 0 for the last 100 epochs

# fac_res_temp = 0.
# fac_smaller = 0.

for run in range(epochs):
    if flag:
        break    
    loss_curr = []
    loss_vali_curr = []
    if (run +1) % 100 == 0:
        fac_res_temp -= fac_smaller
    
    for batch in range(batches):
        # Get data for minibatch
        batch_traj, batch_traj_lag = get_mini_batch_gen(batch_size_gen, batch)
        
        if stochastic_gradient:
            training_dict = {gen_chi_tau: batch_traj,
                             gen_chi_tau2: batch_traj,
                           gen_x_tau: batch_traj_lag,
                           keep_prob_drop : (1.-dropout),
                             lr_tensor : lr_gentemp,
                             fac_res_tf : fac_res_temp
                          }
        else:
            training_dict = {gen_chi_tau: batch_traj,
                             gen_chi_tau2: batch_traj,
                           gen_x_tau: batch_traj_lag,
                           keep_prob_drop : (1.-dropout),
                             fac_res_tf : fac_res_temp
                          }

        # Create Input for Training D
        # Train Dyn
        _ = sess.run(output_nodes_list,
                                        feed_dict=training_dict)
    for batch in range(batches):
        batch_traj, batch_traj_lag = get_mini_batch_gen(batch_size_gen, batch)
        training_dict = {gen_chi_tau: batch_traj,
                         gen_chi_tau2: batch_traj,
                       gen_x_tau: batch_traj_lag,
                       keep_prob_drop : (1.-dropout),
                         fac_res_tf : fac_res_temp
                      }
        loss_temp1 = sess.run(loss_gen, feed_dict=training_dict)
        # Write to Summary
#         validation_dict = {gen_chi_tau : sample_from_dist(chi_X1_vali @ K),
#                            gen_x_tau :  X2_vali,
#                            keep_prob_drop : 1.}
#         loss_vali_temp = sess.run(loss_gen, feed_dict=validation_dict)

#         if (np.isnan(loss_temp1)):
#             print('found NAN in batch {}'.format(batch))
#             flag = True
#             break
            
        loss_curr.append(loss_temp1)
#         loss_vali_curr.append(loss_vali_temp)
        
    loss_np = np.mean(loss_curr)
    if loss_np < best_value :
        best_value = loss_np
#         if conditioned_dist:
#             print('Save cond: {}'.format(best_value))
#             save_path = saver.save(sess, "./saved_models/best_ala_gen_model_test2.ckpt")
#         else:
#             save_path = saver.save(sess, "./saved_models/best_ala_gen_model_test2_equal.ckpt")
#     loss_vali = np.mean(loss_vali_curr)

    # Create a new Summary object with your measure
    summary = tf.Summary()
    summary.value.add(tag="loss_gen_equal", simple_value=loss_np)

    # Add it to the Tensorboard summary writer
    # Make sure to specify a step parameter to get nice graphs over time
    writer.add_summary(summary, run+offset)  
    
    if run % plot_save_every == 0:
        print('Iter: {}'.format(run))
        print('Loss: {:.4}'.format(loss_np))
#         print('Loss vali: {:.4}'.format(loss_vali))
        print()
        
trainings_completed_gen += epochs

In [ ]:
epochs = 201
plot_save_every = 25
offset = trainings_completed_gen

stochastic_gradient = False
lr_gentemp = lr_gen / 100000.

# compute distribution of training data
if stochastic_gradient:
    output_nodes_list = [solver_gen_sg, 
#                      loss_gen,
                          ]
else:
    output_nodes_list = [solver_gen, 
#                      loss_gen,
                          ]
flag = False
# fac_res_temp = 1.
# fac_smaller = 1. /( (epochs // 100) - 1) # so it is 0 for the last 100 epochs

fac_res_temp = 0.
fac_smaller = 0.

for run in range(epochs):
    if flag:
        break    
    loss_curr = []
    loss_vali_curr = []
    if (run +1) % 100 == 0:
        fac_res_temp -= fac_smaller
    
    for batch in range(batches):
        # Get data for minibatch
        batch_traj, batch_traj_lag = get_mini_batch_gen(batch_size_gen, batch)
        
        if stochastic_gradient:
            training_dict = {gen_chi_tau: batch_traj,
                             gen_chi_tau2: batch_traj,
                           gen_x_tau: batch_traj_lag,
                           keep_prob_drop : (1.-dropout),
                             lr_tensor : lr_gentemp,
                             fac_res_tf : fac_res_temp
                          }
        else:
            training_dict = {gen_chi_tau: batch_traj,
                             gen_chi_tau2: batch_traj,
                           gen_x_tau: batch_traj_lag,
                           keep_prob_drop : (1.-dropout),
                             fac_res_tf : fac_res_temp
                          }

        # Create Input for Training D
        # Train Dyn
        _ = sess.run(output_nodes_list,
                                        feed_dict=training_dict)
    for batch in range(batches):
        batch_traj, batch_traj_lag = get_mini_batch_gen(batch_size_gen, batch)
        training_dict = {gen_chi_tau: batch_traj,
                         gen_chi_tau2: batch_traj,
                       gen_x_tau: batch_traj_lag,
                       keep_prob_drop : (1.-dropout),
                         fac_res_tf : fac_res_temp
                      }
        loss_temp1 = sess.run(loss_gen, feed_dict=training_dict)
        # Write to Summary
#         validation_dict = {gen_chi_tau : sample_from_dist(chi_X1_vali @ K),
#                            gen_x_tau :  X2_vali,
#                            keep_prob_drop : 1.}
#         loss_vali_temp = sess.run(loss_gen, feed_dict=validation_dict)

#         if (np.isnan(loss_temp1)):
#             print('found NAN in batch {}'.format(batch))
#             flag = True
#             break
            
        loss_curr.append(loss_temp1)
#         loss_vali_curr.append(loss_vali_temp)
        
    loss_np = np.mean(loss_curr)
    if loss_np < best_value :
        best_value = loss_np
#         if conditioned_dist:
#             print('Save cond: {}'.format(best_value))
#             save_path = saver.save(sess, "./saved_models/best_ala_gen_model_test2.ckpt")
#         else:
#             save_path = saver.save(sess, "./saved_models/best_ala_gen_model_test2_equal.ckpt")
#     loss_vali = np.mean(loss_vali_curr)

    # Create a new Summary object with your measure
    summary = tf.Summary()
    summary.value.add(tag="loss_gen_equal", simple_value=loss_np)

    # Add it to the Tensorboard summary writer
    # Make sure to specify a step parameter to get nice graphs over time
    writer.add_summary(summary, run+offset)  
    
    if run % plot_save_every == 0:
        print('Iter: {}'.format(run))
        print('Loss: {:.4}'.format(loss_np))
#         print('Loss vali: {:.4}'.format(loss_vali))
        print()
        
trainings_completed_gen += epochs

# Calculate cond. probs

In [ ]:
def get_points_in_square(dih, c, l):
    
            
    dht = np.array(dih.T)
    
    xlim = np.array([c[0]-l/2,c[0]+l/2])
    ylim1 = np.array([c[1]-l/2,c[1]+l/2])
    
    if ylim1[1] < np.pi and ylim1[0] > -np.pi:
        filters_num = 1
        ylim2 = None
        
    elif ylim1[0] < -np.pi:
        filters_num = 2
        ylim2  = np.array([2*np.pi + ylim1[0], np.pi])
        ylim1[0] = - np.pi
        
    else:
        ylim2  = np.array([-np.pi, -2*np.pi + ylim1[1]])
        ylim1[1] = np.pi
        filters_num = 2

    conds = np.empty((dht.shape[1], filters_num))

    
    for index_ylim, ylim in enumerate([ylim1, ylim2]):       
        if index_ylim < filters_num:
            
            cond_1 = dht[0] < xlim[0]
            cond_2 = dht[0] > xlim[1]
            
            cond_3 = dht[1] < ylim[0]
            cond_4 = dht[1] > ylim[1]

            conds[:,index_ylim] = np.any([cond_1, cond_2, cond_3, cond_4], axis = 0)
    
    indexes_filt = np.where(np.logical_not(conds))[0]

    return indexes_filt

In [ ]:
dihedral_lag = dihedral[tau:]
gen_cond_points = np.zeros((6), dtype = int)

for r in range(6):
    
    filt_center =  dihedral[state_centers[r]]
    filt_rad = 0.7
    
    dih_center = get_points_in_square(dihedral[:-tau], filt_center, filt_rad)
    gen_cond_points[r] = len(dih_center)
    
    fig, ax = plt.subplots()

    range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

    hist_im = ax.hist2d(*dihedral_lag[dih_center].T, bins=100, range = range_mat, norm=matplotlib.colors.LogNorm(), cmap = plt.cm.viridis, normed = True)
    ax.scatter(*dihedral[state_centers[r]], s=40, c='C3')
    
    e = matplotlib.patches.Rectangle(filt_center-filt_rad/2, filt_rad, filt_rad, color='C1')
    e.set_clip_box(ax.bbox)
    e.set_alpha(0.4)
    ax.add_artist(e)
    
    if r == 5:
        new_fc = filt_center-filt_rad/2 + [0, 2*np.pi]
        e2 = matplotlib.patches.Rectangle(new_fc, filt_rad, filt_rad, color='C1')
        e2.set_clip_box(ax.bbox)
        e2.set_alpha(0.4)
        ax.add_artist(e2)
        
    
    ax.set_xlim(-np.pi,np.pi)
    ax.set_ylim(-np.pi,np.pi)
    font = {'fontsize': 14}
    ax.set_xticks([-3.1,0,3.1])
    ax.set_yticks([-3.1,0,3.1])
    ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
    ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
    ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
    ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
#     fig.savefig('cond_dist_real_state{}.pdf'.format(r), bbox_inches='tight')

#     plt.colorbar(hist_im[-1], ax = ax)
    plt.show()

In [ ]:
grid_points = 6
cond_points = gen_cond_points.max()



predict_cond = np.empty((grid_points, cond_points, input_size))

starting_pools = []

for r in range(grid_points):

    filt_center =  dihedral[state_centers[r]]
    filt_rad = 0.7

    dih_center = get_points_in_square(dihedral[:-tau], filt_center, filt_rad)
    starting_pools.append(dih_center)

In [ ]:
for index_cond in range(cond_points):

    indexes_best_dih = np.empty((grid_points), dtype = int)
    for r in range(grid_points):
        indexes_best_dih[r] = np.random.choice(starting_pools[r])

    temp = sess.run(chi_out, feed_dict = {x_t: traj_ord[indexes_best_dih], keep_prob_drop : 1.})


#     temp_samp = sample_from_dist(temp)
    predict_cond[:,index_cond] = sess.run(gen_x1, feed_dict = {gen_chi_tau: temp, keep_prob_drop : 1.})[:,:]

In [ ]:
mol_gen = md.load_dcd('/group/ag_cmb/scratch/deeptime_data/alanine_data/ala2.dcd', top='/group/ag_cmb/scratch/deeptime_data/alanine_data/ala2.pdb')[:cond_points]
idx_heavy = [a for a in range(mol_gen.n_atoms) if mol_gen.topology.atom(a).element.symbol != 'H']
mol_gen_small = mol_gen.atom_slice(idx_heavy)

In [ ]:
dihedral_ind_small = [[1,3,4,6], [3,4,6,8]]
dihedrals_cond = np.zeros((cond_points, 6, 2))
for i in range(6):
    mol_gen_small.xyz = predict_cond[i].reshape(-1, mol_gen_small.n_atoms, 3)
    dihedrals_cond[:,i] = md.compute_dihedrals(mol_gen_small, dihedral_ind_small)

In [ ]:
offset = 0.5
low = -np.pi - offset
for r in range(6):
    
    filt_center =  dihedral[state_centers[r]]
    filt_rad = 0.7
    
    
    fig, ax = plt.subplots()

    range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

    hist_im = ax.hist2d(*dihedrals_cond[:gen_cond_points[r],r].T, bins=100, range = range_mat, norm=matplotlib.colors.LogNorm(), cmap = plt.cm.viridis, normed = True)
    ax.scatter(*dihedral[state_centers[r]], s=40, c='C3')
    
    e = matplotlib.patches.Rectangle(filt_center-filt_rad/2, filt_rad, filt_rad, color='C1')
    e.set_clip_box(ax.bbox)
    e.set_alpha(0.4)
    ax.add_artist(e)
    
    if r == 5:
        new_fc = filt_center-filt_rad/2 + [0, 2*np.pi]
        e2 = matplotlib.patches.Rectangle(new_fc, filt_rad, filt_rad, color='C1')
        e2.set_clip_box(ax.bbox)
        e2.set_alpha(0.4)
        ax.add_artist(e2)
        
    
    ax.set_xlim(-np.pi,np.pi)
    ax.set_ylim(-np.pi,np.pi)
    font = {'fontsize': 14}
    ax.set_xticks([-3.1,0,3.1])
    ax.set_yticks([-3.1,0,3.1])
    ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
    ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
    ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
    ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
    fig.savefig('cond_dist_gen_state{}.pdf'.format(r), bbox_inches='tight')

#     plt.colorbar(hist_im[-1], ax = ax)
    plt.show()

In [ ]:
grid_points = 1
cond_points = 250000 -1


predict_stat = np.empty((cond_points, 30))
predict_stat[0,:] = traj_whole[0]

for index_cond in range(1, cond_points):
    temp = sess.run(chi_out, feed_dict = {x_t: predict_stat[index_cond-1][None,:], keep_prob_drop : 1.})
#     temp_samp = sample_from_dist(temp)
    predict_stat[index_cond] = sess.run(gen_x1, feed_dict = {gen_chi_tau: temp, keep_prob_drop : 1.})

In [ ]:
mol_gen = md.load_dcd('/group/ag_cmb/scratch/deeptime_data/alanine_data/ala2.dcd', top='/group/ag_cmb/scratch/deeptime_data/alanine_data/ala2.pdb')[:cond_points]
idx_heavy = [a for a in range(mol_gen.n_atoms) if mol_gen.topology.atom(a).element.symbol != 'H']
mol_gen_small = mol_gen.atom_slice(idx_heavy)

In [ ]:
dihedrals_gen = np.zeros((cond_points, 2))

mol_gen_small.xyz = predict_stat.reshape(-1, mol_gen_small.n_atoms, 3)
dihedrals_gen = md.compute_dihedrals(mol_gen_small, dihedral_ind_small)

In [ ]:
fig, ax = plt.subplots()

range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

hist_im = ax.hist2d(dihedrals_gen[:,0], dihedrals_gen[:,1], bins=300, range = range_mat, norm=matplotlib.colors.LogNorm(), cmap = plt.cm.viridis)
ax.scatter(dihedral[state_filt_index,0], dihedral[state_filt_index,1], s=40, c='C3')
ax.set_xlim(-np.pi,np.pi)
ax.set_ylim(-np.pi,np.pi)
if filter_data:
    e = matplotlib.patches.Rectangle(*mplt_filt1, color='C1')
    e.set_clip_box(ax.bbox)
    e.set_alpha(0.2)
    ax.add_artist(e)
    if mplt_filt2:
        e2 = matplotlib.patches.Rectangle(*mplt_filt2, color='C1')
        e2.set_clip_box(ax.bbox)
        e2.set_alpha(0.2)
        ax.add_artist(e2)
font = {'fontsize': 14}
ax.set_xticks([-3.1,0,3.1])
ax.set_yticks([-3.1,0,3.1])
ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
plt.colorbar(hist_im[-1], ax = ax)
# fig.savefig('generated_data_withoutKoop_removed_{}state.pdf'.format(state_to_be_filtered), bbox_inches='tight')
plt.show()

In [ ]:
grid_points = 1
cond_points = length_data


predict_stat = np.empty((cond_points, 30))
predict_stat[0,:] = traj_whole[0]

for index_cond in range(1, cond_points):
    temp = sess.run(chi_out, feed_dict = {x_t: predict_stat[index_cond-1][None,:], keep_prob_drop : 1.})
    temp_samp = sample_from_dist(temp)
    predict_stat[index_cond] = sess.run(gen_x1, feed_dict = {gen_chi_tau: temp_samp, keep_prob_drop : 1.})

In [ ]:
mol_gen = md.load_dcd('/group/ag_cmb/scratch/deeptime_data/ala2.dcd', top='/group/ag_cmb/scratch/deeptime_data/ala2.pdb')[:cond_points]
idx_heavy = [a for a in range(mol_gen.n_atoms) if mol_gen.topology.atom(a).element.symbol != 'H']
mol_gen_small = mol_gen.atom_slice(idx_heavy)

In [ ]:
dihedrals_gen = np.zeros((cond_points, 2))

mol_gen_small.xyz = predict_stat.reshape(-1, mol_gen_small.n_atoms, 3)
dihedrals_gen = md.compute_dihedrals(mol_gen_small, dihedral_ind_small)

In [ ]:
fig, ax = plt.subplots()

range_mat = np.array([[-np.pi,np.pi],[-np.pi,np.pi]])

hist_im = ax.hist2d(dihedrals_gen[:,0], dihedrals_gen[:,1], bins=300, range = range_mat, norm=matplotlib.colors.LogNorm(), cmap = plt.cm.viridis)
ax.scatter(dihedral[state_filt_index,0], dihedral[state_filt_index,1], s=40, c='C3')
ax.set_xlim(-np.pi,np.pi)
ax.set_ylim(-np.pi,np.pi)
e = matplotlib.patches.Rectangle(*mplt_filt1, color='C1')
e.set_clip_box(ax.bbox)
e.set_alpha(0.2)
ax.add_artist(e)
if mplt_filt2:
    e2 = matplotlib.patches.Rectangle(*mplt_filt2, color='C1')
    e2.set_clip_box(ax.bbox)
    e2.set_alpha(0.2)
    ax.add_artist(e2)
font = {'fontsize': 14}
ax.set_xticks([-3.1,0,3.1])
ax.set_yticks([-3.1,0,3.1])
ax.set_xticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_yticklabels([r'-$\pi$','0', r'$\pi$'], fontdict=font)
ax.set_xlabel(r'$\phi$ [rad]', fontsize = 14)
ax.set_ylabel(r'$\psi$ [rad]', fontsize = 14)
plt.colorbar(hist_im[-1], ax = ax)
# fig.savefig('generated_data_withoutKoop_removed_{}state.pdf'.format(state_to_be_filtered), bbox_inches='tight')
plt.show()

In [ ]:
generated_closest_points = np.argsort(np.linalg.norm(dihedrals_gen[:] - dihedral[state_filt_index], axis = 1))[:100]
gen_filter_traj = predict_stat[generated_closest_points]

In [ ]:
mol_gen_fil = md.load_dcd('/group/ag_cmb/scratch/deeptime_data/ala2.dcd', top='/group/ag_cmb/scratch/deeptime_data/ala2.pdb')[:gen_filter_traj.shape[0]+1]
idx_heavy = [a for a in range(mol_gen.n_atoms) if mol_gen.topology.atom(a).element.symbol != 'H']
mol_gen_fil_small = mol_gen_fil.atom_slice(idx_heavy)
mol_gen_fil_small.xyz = np.concatenate([traj_whole[state_filt_index].reshape(-1,10,3), gen_filter_traj.reshape(-1,10,3)], axis = 0)
mol_gen_fil_small.save_pdb('/group/ag_cmb/scratch/deeptime_data/gen_data_withoutKoop_ala_filt_state{}.pdb'.format(state_to_be_filtered))

In [ ]:
closest_structures_index = np.argsort(np.linalg.norm(predict_stat - traj_whole[state_filt_index], axis = 1))[:100]
closest_structures = predict_stat[closest_structures_index]

In [ ]:
mol_gen_fil = md.load_dcd('/group/ag_cmb/scratch/deeptime_data/ala2.dcd', top='/group/ag_cmb/scratch/deeptime_data/ala2.pdb')[:closest_structures.shape[0]+1]
idx_heavy = [a for a in range(mol_gen.n_atoms) if mol_gen.topology.atom(a).element.symbol != 'H']
mol_gen_fil_small = mol_gen_fil.atom_slice(idx_heavy)
mol_gen_fil_small.xyz = np.concatenate([traj_whole[state_filt_index].reshape(-1,10,3), closest_structures.reshape(-1,10,3)], axis = 0)
mol_gen_fil_small.save_pdb('/group/ag_cmb/scratch/deeptime_data/gen_data_withoutKoop_ala_closest_filt_state{}.pdb'.format(state_to_be_filtered))